In [1]:
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

    from google.colab import drive
    drive.mount('/content/gdrive')

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

Mounted at /content/gdrive


In [2]:
import os
import pandas as pd

In [3]:
!ls

gdrive	sample_data


In [4]:
imageDir = "/content/gdrive/MyDrive/ZKML Regens/ml_ready_plants"
annotations_file = "/content/gdrive/MyDrive/ZKML Regens/ml_ready_plants_metadata/annotations_images.csv"

In [5]:
annotationsDF = pd.read_csv(annotations_file)

<ipython-input-5-bb073583a247>:1: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  annotationsDF = pd.read_csv(annotations_file)


In [6]:
unique_order = annotationsDF[['genus', 'order']].drop_duplicates().values.tolist()

order = {}
for o in unique_order:
  order[o[0]] = o[1]

In [7]:
# prompt: get unique genus

unique_genus = annotationsDF['genus'].unique().tolist()


In [8]:
labels = {"Laurales": 0,
          "Fabales": 1,
          "Malpighiales": 2,
          "Sapindales": 3,
          "Myrtales": 4,
          "Arecales": 5}

In [11]:
labels_list = []

def list_files_in_directories(root_dir):
  """Lists files in all directories within a given root directory."""
  for dirpath, dirnames, filenames in os.walk(root_dir):
    if dirpath == imageDir:
      continue

    directory = dirpath.split("/")[-1]
    genus_dir = directory.split("_")[0].split("(")[-1]
    if genus_dir not in unique_genus:
      continue

    for f, file in enumerate(filenames):
      if f % 5 == 1:
        data = "Test"
      else:
        data = "Train"

      image_path = os.path.join(directory, file)
      label = order[genus_dir]
      if label not in labels:
        labelNum = 6
      else:
        labelNum = labels[label]
      labels_list.append([image_path, labelNum, label, data])

list_files_in_directories(imageDir)
df = pd.DataFrame(labels_list, columns=["image_path", "target", "order", "data"])
df

,image_path,target,order,data
0,Passovia_rufa/Passovia rufa - (erva de passari...,6,Santalales,Train
1,Loro aritú (Aniba_ferrea)/DJI_20240604184256_...,0,Laurales,Train
2,Loro aritú (Aniba_ferrea)/DJI_20240604184302_...,0,Laurales,Test
3,Loro aritú (Aniba_ferrea)/DJI_20240604184309_...,0,Laurales,Train
4,Loro aritú (Aniba_ferrea)/DJI_20240604184325_...,0,Laurales,Train
...,...,...,...,...
588,Caryocar_glabrum/Caryocar glabrum - DJI_202405...,2,Malpighiales,Train
589,Caryocar_glabrum/Caryocar glabrum - DJI_202405...,2,Malpighiales,Test
590,Caryocar_glabrum/Caryocar glabrum - DJI_202405...,2,Malpighiales,Train
591,Caryocar_glabrum/Caryocar glabrum -DJI_2024040...,2,Malpighiales,Train


In [12]:
# Get the directory of the annotations_file
annotations_file_dir = os.path.dirname(annotations_file)
df.to_csv(os.path.join(annotations_file_dir, 'imagesOrders.csv'), index=False)